In [1]:
import torch
import torch.nn.functional as F
import sympy as sp
import pandas as pd

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [73]:
df = pd.read_csv('physics_equations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 11):  
        v_name = row.get(f'v{i}_name', None)
        v_low = row.get(f'v{i}_low', None)
        v_high = row.get(f'v{i}_high', None)
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

'''for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()'''

'for i, func in enumerate(functions):\n    print(f"Function {i+1}:")\n    print(f"  Formula: {func[\'formula\']}")\n    print(f"  Number of Variables: {num_vars_per_func[i]}")\n    print("  Variables:")\n    for var in func[\'variables\']:\n        print(f"    - Name: {var[\'name\']}, Range: ({var[\'low\']}, {var[\'high\']})")\n    print()'

In [88]:
def generate_function(function, sample_size, x, device):
    sympy_symbols = []
    param_tensors = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val, max_val = var["low"], var["high"]
        param = (max_val - min_val) * torch.rand(sample_size, 1, device=device) + min_val
        param_tensors.append(param)

    sympy_symbols.append(sp.symbols('x'))
    params = torch.cat(param_tensors, dim=1)
    formula = sp.sympify(function["formula"])
    eval_func = sp.lambdify(sympy_symbols, formula, modules="numpy")
        
    results = []
    for xi in x:
        input_values = torch.cat([params, xi.expand(sample_size, 1)], dim=1)
        results.append(eval_func(*input_values.T))
    results = torch.stack(results, dim=1)
    
    return results, formula, sympy_symbols, params

In [89]:
sample_size = 1000
sequence_length = 100
num_funcs = 10

x_values = torch.linspace(-1, 1, sequence_length).to(device)
hold = []
for f in functions[0:10]:
    try:
        results = generate_function(f, sample_size, x_values, device)
        hold.append(results)
    except Exception as e:
        print(f"Error processing function {f}: {e}")

Error processing function {'formula': 'Q/((m)*(c)*(dT))', 'variables': [{'name': 'Q', 'low': 0.0, 'high': 1}, {'name': 'm', 'low': 0.1, 'high': 1.0}, {'name': 'c', 'low': 0.1, 'high': 1.0}, {'name': 'dT', 'low': 0.1, 'high': 1.0}]}: unsupported operand type(s) for /: 'AssumptionKeys' and 'Mul'


In [93]:
hold_data = [torch.tensor(l[0]) for l in hold]
hold_formulas = [l[1] for l in hold]
hold_symbols = [l[2] for l in hold]
hold_params = [l[3] for l in hold]
hold_num_params = [len(l[2]) for l in hold]

/tmp/ipykernel_3791442/313369855.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hold_data = [torch.tensor(l[0]) for l in hold]


In [96]:
hold_data[0].shape, hold_formulas[0], hold_symbols[0], hold_params[0].shape, hold_num_params[0]

(torch.Size([1000, 100]),
 a*x**2 + b*x + c,
 [a, b, c, x],
 torch.Size([1000, 3]),
 4)

In [97]:
def evaluate_function(params, symbols, formula):
    if params.shape[0] != 1:
        var_values = {symbols[j]: params[j].item() for j in range(len(symbols))}
    else:
        var_values = {symbols[j]: params[:, j, :].item() for j in range(len(symbols))}
    evaluated = formula.subs(var_values)
    return float(evaluated)

In [98]:
def compute_derivative(parameters, func, symbols, formula, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[2]
    gradients = torch.zeros(batch_size, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            perturbed_params_pos = param_tensor.clone()
            perturbed_params_neg = param_tensor.clone()
            
            perturbed_params_pos[0, j, 0] += epsilon
            forward_value = func(perturbed_params_pos, symbols, formula)
            
            perturbed_params_neg[0, j, 0] -= epsilon
            backward_value = func(perturbed_params_neg, symbols, formula)
            
            gradients[i, j] = (forward_value - backward_value) / (2 * epsilon)
    
    return gradients


In [99]:
def compute_hessian(parameters, func, symbols, formula, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[2]
    hessian = torch.zeros(batch_size, num_params, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            for k in range(j, num_params):
                perturbed_params_pp = param_tensor.clone()
                perturbed_params_pp[0, j, 0] += epsilon
                perturbed_params_pp[0, k, 0] += epsilon
                ff_value = func(perturbed_params_pp, symbols, formula)

                perturbed_params_nn = param_tensor.clone()
                perturbed_params_nn[0, j, 0] -= epsilon
                perturbed_params_nn[0, k, 0] -= epsilon
                bb_value = func(perturbed_params_nn, symbols, formula)

                if j == k:
                    center_value = func(param_tensor, symbols, formula)
                    hessian[i, j, k] = (ff_value - 2 * center_value + bb_value) / (epsilon ** 2)
                else:
                    perturbed_params_pn = param_tensor.clone()
                    perturbed_params_pn[0, j, 0] += epsilon
                    perturbed_params_pn[0, k, 0] -= epsilon
                    fb_value = func(perturbed_params_pn, symbols, formula)

                    perturbed_params_np = param_tensor.clone()
                    perturbed_params_np[0, j, 0] -= epsilon
                    perturbed_params_np[0, k, 0] += epsilon
                    bf_value = func(perturbed_params_np, symbols, formula)

                    hessian[i, j, k] = (ff_value - fb_value - bf_value + bb_value) / (4 * epsilon ** 2)
                    hessian[i, k, j] = hessian[i, j, k]

    return hessian

In [19]:
derivative_test = compute_derivative(hold_params[0], evaluate_function, hold_symbols[0], hold_formulas[0])
hessican_test = compute_hessian(hold_params[0], evaluate_function, hold_symbols[0], hold_formulas[0])

In [21]:
hessican_test.shape

torch.Size([1000, 4, 4])

In [8]:
'''def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6): #need to fix
    batch_size = parameters.shape[0]
    max_num_params = parameters.shape[2]
    hessians = torch.zeros((batch_size, max_num_params, max_num_params))
    
    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(max_num_params):
            for k in range(max_num_params):
                # Perturb j-th and k-th parameters
                perturbed_params = param_tensor.clone()
                
                # Compute f(x + epsilon * e_j + epsilon * e_k)
                perturbed_params_jk = perturbed_params.clone()
                perturbed_params_jk[:, j, :] += epsilon
                perturbed_params_jk[:, k, :] += epsilon
                f_plus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x + epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_plus_minus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j + epsilon * e_k)
                perturbed_params_jk[:, j, :] -= 2 * epsilon
                perturbed_params_jk[:, k, :] += 2 * epsilon
                f_minus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_minus_minus = func(perturbed_params_jk, symbols, formula)
                hessians[i, j, k] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * epsilon**2)
    
    return hessians'''

In [10]:
derivatives = [compute_derivative(hold_params[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(hold_params))]
hessians = [compute_hessians(hold_params[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(hold_params))]

In [42]:
flat_result = torch.flatten(torch.stack(hold_data).unsqueeze(2), start_dim=0, end_dim=1)

max_der = max(d.size(1) for d in derivatives)
padded_ders = []
for d in derivatives:
    padded_tensor = F.pad(d, pad=(0, max_der - d.size(1)))
    padded_ders.append(padded_tensor)
flat_der = torch.flatten(torch.stack(padded_ders), start_dim=0, end_dim=1)

max_hess = max_der**2
padded_hess = []
for h in hessians:
    h = torch.flatten(h, start_dim=1)
    padded_tensor = F.pad(h, pad=(0, max_hess - h.size(1)))
    padded_hess.append(padded_tensor)
flat_hess = torch.flatten(torch.stack(padded_hess), start_dim=0, end_dim=1)

flat_der = F.pad(flat_der, (0, flat_hess.size(-1) - flat_der.size(-1)))
flat_result = F.pad(flat_result, (0, flat_hess.size(-1) - flat_result.size(-1)))

flat_result.shape, flat_der.shape, flat_hess.shape

(torch.Size([10000, 81]), torch.Size([10000, 81]), torch.Size([10000, 81]))

In [44]:
torch.save({
    'flattened_data': torch.stack([flat_result, flat_der, flat_hess], dim=2),
    'results': hold_data,
    'formulas': hold_formulas,
    'symbols': hold_symbols,
    'params': hold_params,
    'num_params': hold_num_params,
}, 'hold_data.pth')